<a href="https://colab.research.google.com/github/jeonghyunwoo/nlp_paper/blob/master/naver_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install konlpy

In [0]:
!pip install newspaper3k

In [0]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import newspaper
from newspaper import Article
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from urllib.parse import quote_plus

In [0]:
def mkurl(검색어,시작기사번호,begdate=20190601,enddate=20190709):
    qrytxt = quote_plus(검색어)
    begdate, enddate = str(begdate), str(enddate)
    begd = begdate[:4]+"."+begdate[4:6]+"."+begdate[6:]
    endd = enddate[:4]+"."+enddate[4:6]+"."+enddate[6:]
    url0 = "https://search.naver.com/search.naver?&where=news&query={qry}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds}&de={de}&docid=&nso=so:r,p:from{begdate}to{enddate},a:all&mynews=0&cluster_rank=29&start={pg}&refresh_start=0"
    url = url0.format(qry = qrytxt, ds = begd, de = endd, begdate= begdate, enddate=enddate,pg = 시작기사번호)
    return url

def scrap(url):
    obj = soup(urlopen(url),'lxml')
    src = obj.select('._sp_each_title')
    link = [tag['href'] for tag in src]
    titles = [tag.text for tag in src]
    txts = []
    dats = []
    for lk in link:
        a = Article(lk)
        try:
          a.download();a.parse()
          dats.append(a.publish_date)
          txts.append(a.text)
        except:
          dats.append('')
          txts.append('')
    return pd.DataFrame({'title':titles,
                        'date':dats,
                        'news':txts})

def newsrun(검색어,페이지수,시작날짜,끝날짜):
    '''
    ex) newsrun(검색어='가계부채',페이지수=10,시작날짜=20190601,끝날짜=20190709)
    '''
    newsdfs = []
    for i in np.arange(1,페이지수*10,10):
        url = mkurl(검색어,i,시작날짜,끝날짜)
        newsdfs.append(scrap(url))
        if i % 10 == 0:
          print(f'{i} page done')
    return pd.concat(newsdfs,axis=0)    

In [0]:
%%time
newses = list(map(lambda x: newsrun(x,5,20190601,20190709), ['가계부채','취약계층 채무']))
newsdf = pd.concat(newses, axis=0)
newsdf.to_pickle('newdf.pkl')

CPU times: user 17.6 s, sys: 291 ms, total: 17.9 s
Wall time: 2min 52s


In [0]:

newsdf = pd.read_pickle('newdf.pkl')

In [0]:
newsdf.head()

,title,date,news
0,"홍남기 이어 최종구도 금리인하 압박…""가계부채·집값 걱정말라""",2019-07-08 00:00:00+09:00,[이데일리 김정현 기자] 경제수장들이 잇따라 한국은행 기준금리 인하를 압박하고 나섰...
1,"[인터뷰]박정만 ""가계부채 해결없이 文정부 소득주도성장 불가능""",2019-07-08 10:29:47+09:00,"박정만 서울금융복지상담센터장 인터뷰서 밝혀\n\n""금융은 맑은 날 우산 빌려주고 비..."
2,최종구 “가계부채 안정 가장 잘 했다”,None,
3,"한국, 가계부채에 이어 국가채무 논쟁…어떻게 봐야 하나 [국제경제읽기 한상...",None,"""돈의 향연이 끝나고 반란이 시작된다"". 5년 전 `머니 볼`의 저자인 마이클 루이..."
4,"취임 2년 최종구, “가계부채 안정 최고 성과..재정확대 정책 필요”",None,\n\n최종구 금융위원장이 지난 5일 서울 광화문에서 열린 취임 2주년 기자 간담회...
